In [4]:
PATH = '/home/g056122/FeatExt_Data_Clustering/final_github/content_based_filtering'

In [5]:
DATAPATH = '/home/g056122/ISG_studienarbeit/Movielens_100K'

In [6]:
''' 
Load 'validate', 'euclid' and 'valres' module available from 'Module' folder
'''
# import required libraries
import numpy as np
import pandas as pd
import datetime
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from validate import evaluation
from euclid import EuclideanDistance
from valres import result
from sklearn.model_selection import KFold

In [7]:
# import movies dataset
dfmovies = pd.read_csv("/home/g056122/ISG_studienarbeit/Movielens_100K/u.item", sep='|', header=None, engine='python', encoding='latin-1')
dfmovies.columns = ['movieId', 'movietitle', 'releasedate', 'videoreleasedate','imdburl', 'unknown', 'action', 'adventure', 'animation','childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film-Noir', 'horror', 'musical', 'mystery', 'romance', 'sci-fi','thriller', 'war', 'western']

In [8]:
# drop features of movies dataset
dfmovies = dfmovies.drop(['videoreleasedate', 'imdburl'], axis=1)

In [9]:
# transform releasedate feature
dfmovies['releasedate'] = pd.to_datetime (dfmovies['releasedate'], format='%d-%b-%Y')
dfmovies['year']=dfmovies['releasedate'].dt.year

In [10]:
# drop features of movies dataset
dfmovies = dfmovies.drop(['movietitle', 'releasedate'], axis=1)

In [11]:
dfmovies.head()

,movieId,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,...,film-Noir,horror,musical,mystery,romance,sci-fi,thriller,war,western,year
0,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995.0
1,2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
3,4,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1995.0
4,5,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,1995.0


In [12]:
# encode features of movies dataset
n_dfmovies = pd.get_dummies(dfmovies, columns = ['year'])

In [13]:
n_dfmovies.head()

,movieId,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,...,year_1989.0,year_1990.0,year_1991.0,year_1992.0,year_1993.0,year_1994.0,year_1995.0,year_1996.0,year_1997.0,year_1998.0
0,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,4,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,5,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0


In [14]:
n_dfmovies.describe()

,movieId,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,...,year_1989.0,year_1990.0,year_1991.0,year_1992.0,year_1993.0,year_1994.0,year_1995.0,year_1996.0,year_1997.0,year_1998.0
count,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,...,1682.000000,1682.000000,1682.00000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000
mean,841.500000,0.001189,0.149227,0.080262,0.024970,0.072533,0.300238,0.064804,0.029727,0.431034,...,0.008918,0.014269,0.01308,0.021998,0.074911,0.127229,0.130202,0.211058,0.170036,0.038644
std,485.695893,0.034473,0.356418,0.271779,0.156081,0.259445,0.458498,0.246253,0.169882,0.495368,...,0.094041,0.118632,0.11365,0.146719,0.263326,0.333329,0.336626,0.408181,0.375776,0.192803
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,421.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,841.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1261.750000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1682.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
categoryindex = list(range(1,n_dfmovies.shape[1]))

In [16]:
# KPrototype clustering
num_clusters = 8
kprototype = KPrototypes(n_clusters=num_clusters, init = "Huang")
kprototype.fit_predict(n_dfmovies, categorical = categoryindex)    

array([3, 3, 3, ..., 5, 5, 5], dtype=uint16)

In [17]:
# cluster centroids
centroids = kprototype.cluster_centroids_

In [18]:
centroids.dtype, centroids.shape

(dtype('float64'), (8, 91))

In [19]:
n_dfmovies = n_dfmovies.values

In [20]:
n_dfmovies.shape

(1682, 91)

In [21]:
# compute euclidean distance between a cluster centroid and each data point
euc = EuclideanDistance(n_dfmovies, num_clusters, centroids)
eucres = euc.compute()

In [22]:
eucres.shape

(1682, 8)

In [24]:
val_m = np.array(dfmovies.movieId.values)
val_m = val_m.reshape(eucres.shape[0],1)
val_m.shape

(1682, 1)

In [25]:
# merge dataset
n_dfmovies = np.concatenate((val_m, n_dfmovies), axis=1)
n_dfmovies, n_dfmovies.shape

(array([[   1,    1,    0, ...,    0,    0,    0],
        [   2,    2,    0, ...,    0,    0,    0],
        [   3,    3,    0, ...,    0,    0,    0],
        ...,
        [1680, 1680,    0, ...,    0,    0,    1],
        [1681, 1681,    0, ...,    0,    0,    0],
        [1682, 1682,    0, ...,    1,    0,    0]]),
 (1682, 92))

In [27]:
n_dfmovies = np.concatenate((n_dfmovies, eucres), axis=1)
n_dfmovies.shape

(1682, 100)

In [28]:
n_dfmovies

array([[1.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        1.57450159e+03, 3.14006369e+02, 7.30502738e+02],
       [2.00000000e+00, 2.00000000e+00, 0.00000000e+00, ...,
        1.57350159e+03, 3.13006390e+02, 7.29502742e+02],
       [3.00000000e+00, 3.00000000e+00, 0.00000000e+00, ...,
        1.57250095e+03, 3.12003205e+02, 7.28501373e+02],
       ...,
       [1.68000000e+03, 1.68000000e+03, 0.00000000e+00, ...,
        1.04509569e+02, 1.36500110e+03, 9.48501581e+02],
       [1.68100000e+03, 1.68100000e+03, 0.00000000e+00, ...,
        1.05514217e+02, 1.36600073e+03, 9.49501053e+02],
       [1.68200000e+03, 1.68200000e+03, 0.00000000e+00, ...,
        1.06504695e+02, 1.36700073e+03, 9.50501052e+02]])

In [29]:
# import ratings dataset
dfratings = pd.read_csv(f"{DATAPATH}/u.data", sep='\t', header=None, engine='python', encoding='latin-1')
dfratings.columns = ['userId', 'movieId', 'rating', 'timestamp']

In [30]:
# transform feature
dfratings['timestamp'] = pd.to_datetime (dfratings['timestamp'], unit='s')
dfratings['year']=dfratings['timestamp'].dt.year

In [31]:
# scalerize features
scaler = MinMaxScaler()
dfratings[['year', 'rating']]=scaler.fit_transform(dfratings[['year', 'rating']])
dfratings = dfratings.drop('timestamp', axis=1)

In [32]:
# split the dataset to X (features) and y(labels)
n_dfratings = dfratings.drop('rating', axis=1).values
n_ratings = dfratings['rating'].values

In [33]:
n_dfratings

array([[1.96e+02, 2.42e+02, 0.00e+00],
       [1.86e+02, 3.02e+02, 1.00e+00],
       [2.20e+01, 3.77e+02, 0.00e+00],
       ...,
       [2.76e+02, 1.09e+03, 0.00e+00],
       [1.30e+01, 2.25e+02, 0.00e+00],
       [1.20e+01, 2.03e+02, 0.00e+00]])

In [34]:
# merge dataset
list =[]
for m in range(n_dfratings.shape[0]):
    for n in range(n_dfmovies.shape[0]):
        if (n_dfratings[m][1] == n_dfmovies[n][0]):
            merge = np.concatenate((n_dfmovies[n], n_dfratings[m]))
            list.append(merge)
list = np.array(list)
new_list = []
# delete duplicate movieId
for k in range(list.shape[0]):
    val = np.delete(list[k], 0)
    new_list.append(val)
new_list = np.array(new_list)

In [35]:
new_list.shape

(100000, 102)

In [36]:
# split X, y
X = new_list
y = n_ratings

In [37]:
# kfold cross validation
K = evaluation( X, y, 5, 100)
cv = K.kfold()

Fold 1:
Fold 2:
Fold 3:
Fold 4:
Fold 5:


In [38]:
#split tuple 
classes, eucdis = cv
classes, eucdis

(array([[1.28, 2.14, 2.69, ..., 3.67, 2.65, 2.96],
        [4.73, 3.5 , 4.22, ..., 3.64, 2.88, 4.02],
        [2.42, 4.45, 3.72, ..., 3.96, 2.29, 2.48],
        [4.16, 3.79, 2.81, ..., 3.3 , 2.38, 4.18],
        [3.8 , 4.18, 4.08, ..., 3.62, 3.32, 3.92]]),
 array([[0.  , 0.5 , 0.5 , ..., 0.75, 0.5 , 0.25],
        [0.75, 1.  , 0.5 , ..., 0.25, 0.5 , 1.  ],
        [0.5 , 0.75, 1.  , ..., 0.  , 0.  , 0.  ],
        [0.5 , 0.25, 1.  , ..., 0.5 , 0.25, 0.5 ],
        [0.5 , 0.  , 0.25, ..., 0.75, 0.75, 0.5 ]]))

In [39]:
# metrics
ans = result(classes, eucdis)
Metrics = ans.validate()

Metric | Mean | Standard Deviation
RMSE 2.9654995626544953, 0.0026120673802190637
MAE 2.8919725000000005, 0.0026690581859524667
